In [1]:
import pandas as pd
import numpy as np
import seaborn  as sns 
import matplotlib.pyplot as plt
import numpy as np
import folium
import seaborn as sns
import re
# sns.set()
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

import statistics as stat
from shapely.geometry import MultiPoint
from geopy.distance import great_circle
import folium
pd.set_option('display.max_columns', None)

In [2]:
feb_queens = pd.read_csv("../data_skripsi/data_feb_just_queens.csv")

In [3]:
feb_queens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11169 entries, 0 to 11168
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_datetime    11169 non-null  object 
 1   pickup_longitude   11169 non-null  float64
 2   pickup_latitude    11169 non-null  float64
 3   pickup_hour        11169 non-null  int64  
 4   pickup_day         11169 non-null  int64  
 5   pickup_dayname     11169 non-null  object 
 6   pickup_monthname   11169 non-null  object 
 7   pickup_groupofday  11169 non-null  object 
 8   pickup_timeofday   11169 non-null  object 
 9   distance           11169 non-null  float64
 10  average_speed      11169 non-null  float64
 11  duration_min       11169 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 1.0+ MB


In [4]:
feb_queens.duplicated().sum()

0

In [5]:
feb_queens.isna().sum()

pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
pickup_hour          0
pickup_day           0
pickup_dayname       0
pickup_monthname     0
pickup_groupofday    0
pickup_timeofday     0
distance             0
average_speed        0
duration_min         0
dtype: int64

In [6]:
def cluster_pickup(eps, minpts):
    df_pickup_pc = feb_queens[['pickup_latitude', 'pickup_longitude']]
    df_pickup_pc = np.radians(df_pickup_pc)
    df_pickup_pc.astype('float32')
    
    kms_per_radian = 6371
    epsilon = eps / kms_per_radian
    clusters = DBSCAN(eps=epsilon, min_samples=minpts, algorithm='ball_tree', metric='haversine').fit(df_pickup_pc)
    label = clusters.labels_

    print(f'Number of clusters found: {len(np.unique(label[label!=-1]))}')
    print(f'Number of outliers found: {len(label[label==-1])}')

    print(f'The average silhoutte score is: {silhouette_score(df_pickup_pc, label, metric="haversine")}')
    
    df_pickup_pc['cluster_pickup'] = label
    df_pickup_pc[['pickup_latitude', 'pickup_longitude']] = feb_queens[['pickup_latitude', 'pickup_longitude']]
#     
    
    return df_pickup_pc

In [29]:
cluster_p = cluster_pickup(1,11)

Number of clusters found: 2
Number of outliers found: 103
The average silhoutte score is: 0.8060398500917948


In [30]:
for i in range(-1, len(set(cluster_p['cluster_pickup']))):
    print(i)
    count_cluster = cluster_p[cluster_p['cluster_pickup']== i].count()
    print(count_cluster)

-1
pickup_latitude     103
pickup_longitude    103
cluster_pickup      103
dtype: int64
0
pickup_latitude     7692
pickup_longitude    7692
cluster_pickup      7692
dtype: int64
1
pickup_latitude     3374
pickup_longitude    3374
cluster_pickup      3374
dtype: int64
2
pickup_latitude     0
pickup_longitude    0
cluster_pickup      0
dtype: int64


In [79]:
import plotly.express as px
mapbox = px.scatter_mapbox(cluster_p,
                   lon='pickup_longitude',
                   lat='pickup_latitude',
                   color='cluster_pickup',
                   zoom=11,
                   width=1000,
                   height=500,
                  )
mapbox.update_layout(mapbox_style="open-street-map")
mapbox.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
mapbox.show()

In [7]:
cluster_p['cluster_pickup'].unique()

array([ 0,  1,  2, -1,  3], dtype=int64)

In [31]:
merge_cluster = pd.concat([feb_queens, cluster_p['cluster_pickup']], axis=1)
merge_cluster

,pickup_datetime,pickup_longitude,pickup_latitude,pickup_hour,pickup_day,pickup_dayname,pickup_monthname,pickup_groupofday,pickup_timeofday,distance,average_speed,duration_min,cluster_pickup
0,2016-02-23 12:16:42,-73.862694,40.768990,12,1,Tuesday,February,Weekday,Afternoon,13.519613,23.298519,35.0,0
1,2016-02-17 00:07:44,-73.782387,40.644512,0,2,Wednesday,February,Weekday,Late night,22.001279,30.866954,43.0,1
2,2016-02-08 15:43:03,-73.863235,40.770039,15,0,Monday,February,Weekday,Afternoon,14.377826,18.034904,48.0,0
3,2016-02-29 10:04:11,-73.786102,40.639755,10,0,Monday,February,Weekday,Morning,20.266747,21.811745,56.0,1
4,2016-02-12 15:20:26,-73.870949,40.773811,15,4,Friday,February,Weekday,Afternoon,12.384033,14.526725,51.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,2016-02-23 18:52:21,-73.873108,40.774071,18,1,Tuesday,February,Weekday,Evening,8.321224,21.007298,24.0,0
11165,2016-02-29 14:26:54,-73.788361,40.641460,14,0,Monday,February,Weekday,Afternoon,4.587293,12.241848,22.0,1
11166,2016-02-03 23:46:36,-73.788040,40.641788,23,2,Wednesday,February,Weekday,Late night,17.421643,55.699748,19.0,1
11167,2016-02-26 23:36:56,-73.874443,40.773991,23,4,Friday,February,Weekday,Late night,4.053945,17.169648,14.0,0


In [32]:
merge_cluster.to_csv("../data_skripsi/cluster_feb_queens.csv", index=False)

In [33]:
cluster_clean = merge_cluster[merge_cluster['cluster_pickup']!=-1]

In [34]:
cluster_mayor = cluster_clean[cluster_clean['cluster_pickup']== 0].reset_index(drop=True)
cluster_mayor 

,pickup_datetime,pickup_longitude,pickup_latitude,pickup_hour,pickup_day,pickup_dayname,pickup_monthname,pickup_groupofday,pickup_timeofday,distance,average_speed,duration_min,cluster_pickup
0,2016-02-23 12:16:42,-73.862694,40.768990,12,1,Tuesday,February,Weekday,Afternoon,13.519613,23.298519,35.0,0
1,2016-02-08 15:43:03,-73.863235,40.770039,15,0,Monday,February,Weekday,Afternoon,14.377826,18.034904,48.0,0
2,2016-02-12 15:20:26,-73.870949,40.773811,15,4,Friday,February,Weekday,Afternoon,12.384033,14.526725,51.0,0
3,2016-02-23 18:59:13,-73.872925,40.774048,18,1,Tuesday,February,Weekday,Evening,9.842214,13.759988,43.0,0
4,2016-02-29 12:39:38,-73.863533,40.770050,12,0,Monday,February,Weekday,Afternoon,9.128552,15.791824,35.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7687,2016-02-18 19:13:38,-73.863342,40.769909,19,3,Thursday,February,Weekday,Evening,11.676505,21.251474,33.0,0
7688,2016-02-16 00:16:07,-73.862823,40.768951,0,1,Tuesday,February,Weekday,Late night,12.656647,31.101657,24.0,0
7689,2016-02-23 18:52:21,-73.873108,40.774071,18,1,Tuesday,February,Weekday,Evening,8.321224,21.007298,24.0,0
7690,2016-02-26 23:36:56,-73.874443,40.773991,23,4,Friday,February,Weekday,Late night,4.053945,17.169648,14.0,0


In [35]:
cluster_mayor.to_csv("../data_skripsi/cluster_mayor_feb_queens.csv", index=False)

In [36]:
#Hasil statistik dari clustering
cluster = merge_cluster[merge_cluster['cluster_pickup'] == 0]
cluster.mean()

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_20384/3140801943.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cluster.mean()


pickup_longitude   -73.882515
pickup_latitude     40.766618
pickup_hour         13.770021
pickup_day           2.735829
distance             7.987897
average_speed       22.317580
duration_min        29.007280
cluster_pickup       0.000000
dtype: float64

In [ ]:
#ADA 10 CLUSTER 
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

map_osm = folium.Map(location=[40.730610, -73.935242], zoom_start=11, tiles="OpenStreetMap")

for var_descr, var_names, color, icon in zip(
        ["Pickup location"],
        [["pickup_latitude", "pickup_longitude"]],
        ["blue"], ["arrow-up"]):
    feb_copy = feb_queens.copy()  # Let's leave the original untouched
    coords = feb_copy[var_names].round(4)
    coords = feb_copy[var_names].drop_duplicates()
    coords = np.radians(coords)
    
    print("Kept %.1f %% of the %s data." % (100 * coords.shape[0] / feb_copy.shape[0],
                                            var_descr.lower()))
    
    kms_per_radian = 6371
    epsilon = 0.6 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=16, algorithm='ball_tree', metric='haversine').fit(coords)

    # There is one cluster for noisy examples, labeled as -1 
    n_clusters = (len(set(db.labels_)) - 1 if -1 in db.labels_ else
                  len(set(db.labels_)))
    
    coords[var_names] = feb_queens[var_names]
    
    print('Found {} {} clusters.\n-----------------------------------'.format(
        n_clusters, var_descr.lower()))
    #cluster_coords = udah ada isinya dari cluster yg diurutkan dari cluster 0-27
    clusters_coords = [coords[db.labels_ == n] for n in range(n_clusters)]

    print(f'The average silhoutte score is: {silhouette_score(coords, db.labels_)}')
    
    # Find the point in each cluster that is closest to its centroid
    #mencari titik pusat cluster terdekat -> 
    #https://stackoverflow.com/questions/64814999/getting-the-center-point-of-a-cluster-for-latitude-and-longitude-in-python
    centermost_points = [get_centermost_point(list(c.itertuples(index=False))) for
                         c in clusters_coords]
    
    
    #keknya kosong karena antara titik pickup dropoff beda, karena di cluster coords (mode radian) sdgnkan dari jan_copy (titik asli)
    #jadi tidak bisa di merge/tidak ada data yg sama
    clusters_with_all_orig_data = [pd.merge(feb_copy, c, how='inner', on=var_names)
                                   for c in clusters_coords]
    
    for i, ((lat, lng), c) in enumerate(zip(centermost_points, clusters_with_all_orig_data)):
            print('jumlah cluster', c.shape[0])
        
            popup = folium.Popup(folium.IFrame(html="""
            <head>
              <style> body {{ font-family: sans-serif, helvetica, arial; font-size: 14px; }} </style>
            </head>
            <body>
              <p><b>{var_name} cluster no.</b> {cluster_ix}</p>
              <p><b>Nr. of examples in cluster</b>: {n_examples}</p>
              <p><b>Most number of passengers</b>: {most_pass}</p>
              <p><b>Average number of passengers</b>: {avg_pass}</p>
              <p><b>Average trip duration (minutes)</b>: {avg_trip_dur}</p>
              <p><b>Average distance</b>: {distance_trip}</p>
              <p><b>Most common {pickup_or_dropoff} hour</b>: {pickup_h}</p>
              <p><b>Most common {pickup_or_dropoff} day</b>: {pickup_day}</p>
              <p><b>Most common {pickup_or_dropoff} time</b>: {pickup_time}</p>
              <p><b>Most common {pickup_or_dropoff} group of day</b>: {pickup_groupday}</p>
            </body>
            """.format(
                var_name=var_descr, cluster_ix=i + 1, n_examples=c.shape[0],
                most_pass=stat.mode(c['passenger_count']),
                avg_pass=np.round(c["passenger_count"].mean(), 1),
                avg_trip_dur=np.round((c['duration_min']).mean(), 1),
                distance_trip = np.round((c['distance']).mean(), 2),
                pickup_or_dropoff=var_names[0].split('_')[0],
                pickup_h=stat.mode(c['pickup_hour']),
                pickup_day=stat.mode(c['pickup_dayname']),
                pickup_groupday=stat.mode(c['pickup_groupofday']),
                pickup_time=stat.mode(c['pickup_timeofday'])),             
                width=290, height=165))
            [folium.Marker([lat, lng], popup=popup, icon=folium.Icon(color=color, icon=icon)).add_to(map_osm)]
                             
map_osm  